In [2]:
# Import it in colab
!pip install --upgrade pymupdf -q

import fitz  # PyMuPDF
import re

meta_data = {
    "1": "https://swabhs.com/f24-csci544-appliednlp/assets/slides/24f-csci544-appliednlp-lec01-introduction.pdf", 
    "2": "https://swabhs.com/f24-csci544-appliednlp/assets/slides/24f-csci544-appliednlp-lec02-ngrams.pdf", 
    "3": "https://swabhs.com/f24-csci544-appliednlp/assets/slides/24f-csci544-appliednlp-lec03-ngrams-contd-textcat.pdf",
    "4": "https://swabhs.com/f24-csci544-appliednlp/assets/slides/24f-csci544-appliednlp-lec04-logistic-regression.pdf",
    "5" : "https://swabhs.com/f24-csci544-appliednlp/assets/slides/24f-csci544-appliednlp-lec05-word-embeddings.pdf",
    "6" : "https://swabhs.com/f24-csci544-appliednlp/assets/slides/24f-csci544-appliednlp-lec06-word-embeddings-contd.pdf",
}

lec = str(input("Enter the Lecture Number:"))
assert lec in meta_data.keys(), f"Please enter only the number of the lecture pdf you want to fix"
url = meta_data[lec]
print(f"Downloading {url}")
!wget "$url"

pdf_name = url.split("/")[-1]
pdf_path = f"/content/{pdf_name}"
new_pdf_path = f"/content/flattened {pdf_name}"

# Open the PDF document
doc = fitz.open(pdf_path)

# Dictionary to store pages with valid numbers
valid_pages = {}

prev_num = 1
# Iterate through each page in the document
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)  # Get the page
    width, height = page.rect.width, page.rect.height

    # Define the bottom-left corner
    crop_box_left = fitz.Rect(0, height * 0.9, width * 0.05, height)
    crop_box_center = fitz.Rect(width * 0.48, height * 0.9, width * 0.52, height)

    # Extract text from the bottom-left corner
    text = page.get_text("text", clip=crop_box_left)
    text = text.strip()
    if text == "":
      text = str(prev_num)
    else:
      prev_num = int(re.sub(r'[^0-9]', '', text)) + 1 

    valid_pages[text] = page_num + 1

# Print the kept page numbers and their last occurrences
for page_number, slide_number in valid_pages.items():
    print(f"Page Number: {page_number} on Slide Number: {slide_number}")


# Create a new PDF document
new_pdf = fitz.open()

# Add valid pages to the new PDF
for page_number, slide_number in valid_pages.items():
    slide_number = int(slide_number) - 1
    new_pdf.insert_pdf(doc, from_page=slide_number, to_page=slide_number)  # Copy only the valid page

# Save the new PDF file
new_pdf.save(new_pdf_path)
new_pdf.close()

zsh:1: command not found: wget


FileNotFoundError: no such file: '/content/24f-csci544-appliednlp-lec01-introduction.pdf'